## Аннотация: выбор локации для скважины

На основании предоставленных проб нефти в трех регионах, необходимо выявить из них тот, что принесет наибольшую прибыль при бурении новых скважин. 

**Цель** исследования: построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль при наименьшем риске убытков. 

**Задача**: проанализировать возможную прибыль и риски техникой Bootstrap и выбрать регион с вероятностью убытков менее 2,5%.

## Знакомство с данными 

In [65]:
#импортируем библиотеки, которые понадобятся нам в ходе исследования
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [66]:
#применяем специальный метод для чтения файлов формата csv
region_one = pd.read_csv ('/datasets/geo_data_0.csv')
region_two = pd.read_csv ('/datasets/geo_data_1.csv')
region_three = pd.read_csv ('/datasets/geo_data_2.csv')

Теперь необходимо ознакомиться с имеющимися данными, провести их первичный анализ, чтобы получить общее представление о том, с чем придется работать в ходе исследования, и чтобы выявить проблемы в данных, которые нужно будет устранить на этапе предобработки. 

Так как в этот раз мы будем работать сразу с несколькими исходными таблицами, то удобнее будет автоматизировать процесс получения необходимой информации. Создадим функцию, которая будет выводить:
- общую информацию о таблице;
- 10 первых и последних строк фрейма;
- описательную статистику таблицы;
- результаты проверки на наличие в данных явных дубликатов.

In [67]:
def study_of_initial_data(data): #создаем функцию
    print(data.info())
    print()
    display(data.head(10))
    print()
    display(data.tail(10))
    print()
    display(data.describe())
    print()
    print(data.duplicated().sum())

In [68]:
#применяем функцию study_of_initial_data() к каждой таблице
study_of_initial_data(region_one)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None



,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


,id,f0,f1,f2,product
99990,uQHju,-0.169592,0.988432,5.153578,121.253686
99991,5aJ79,-0.880711,0.173065,6.962537,152.614843
99992,vgK5D,0.637601,0.751503,2.918322,95.180855
99993,sOhIk,-0.176367,0.912145,5.241132,89.619662
99994,hWIah,1.562007,-0.311641,1.315442,50.092191
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764
99999,1CWhH,1.764754,-0.266417,5.722849,149.633246


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347



0


In [69]:
study_of_initial_data(region_two)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None



,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


,id,f0,f1,f2,product
99990,6HSBo,4.280434,-1.692187,1.011469,26.953261
99991,obwpN,2.233158,-1.970310,1.004117,26.953261
99992,BYhdw,-2.150114,-13.278717,3.002298,84.038886
99993,FOKH6,-8.421251,-2.813906,4.997337,137.945408
99994,VfVub,-8.646179,-3.661656,-0.003027,3.179103
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364
99999,relB0,-3.426139,-7.794274,-0.003299,3.179103


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408



0


In [70]:
study_of_initial_data(region_three)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None



,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


,id,f0,f1,f2,product
99990,QlYmx,-1.182179,2.127844,2.338479,78.502786
99991,AAcyW,1.570887,-2.430724,5.733563,83.577214
99992,yDrAl,-2.365583,0.148640,2.062216,151.374166
99993,uHI1Q,2.633658,1.873285,-0.498951,33.627420
99994,B2U80,-1.889398,0.851823,6.805412,96.866615
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253
99999,V9kWn,-2.551421,-2.025625,6.090891,102.775767


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838



0


### Вывод:

Мы изучили три датафрейма с исходными данными, каждый из них содержит 100000  строк и 5 столбцов. 
Пропуски и явные дубликаты в данных не были выявлены. Однако для задачи исследования, с большой долей вероятности, не понадобятся идентификационные номера скважин. Удалим соответсвующие столбцы в следующем этапе в ходе подготовки данных для машинного обучения.

## Подготовка данных для машинного обучения

Подготовим имеющиеся данные, чтобы обучить с их помощью модель машинного обучения, а затем проверить ее работу. Для начала удалим избыточный столбец с id скважин в каждой из таблиц:

In [71]:
region_one = region_one.drop(['id'], axis = 1)
region_two = region_two.drop(['id'], axis = 1)
region_three = region_three.drop(['id'], axis = 1)

Теперь разделим данные каждой таблицы на признаки и целевой признак, на обучающую и валидационную выборки (в соотношении 75:25, согласно условию). Создадим для этих целей функцию preparing_for_ml:

In [72]:
def preparing_for_ml(data): 
    global target_train, features_train, target_valid, features_valid
    target = data['product']
    features = data.drop(['product'], axis = 1)
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, 
                                                                                  test_size = 0.25, 
                                                                                  random_state = 12345)

Теперь применим созданную функцию к имеющимся таблицам. Чтобы отделить выборки каждого региона и не перепутать их, после каждого запуска функции будем переименовывать получившиеся выборки в соответствие с регионами.

In [73]:
preparing_for_ml(region_one) #применяем функцию

#переименовываем выборки
features_train_one, features_valid_one, target_train_one, target_valid_one = features_train, features_valid, target_train, target_valid

preparing_for_ml(region_two)
features_train_two, features_valid_two, target_train_two, target_valid_two = features_train, features_valid, target_train, target_valid

preparing_for_ml(region_three)
features_train_three, features_valid_three, target_train_three, target_valid_three = features_train, features_valid, target_train, target_valid

#выводим на экран размеры получившихся выборок
print(features_train_one.shape, features_train_two.shape, features_train_three.shape)
print(features_valid_one.shape, features_valid_two.shape, features_valid_three.shape)
print(target_train_one.shape, target_train_two.shape, target_train_three.shape)
print(target_valid_one.shape, target_valid_two.shape, target_valid_three.shape)

(75000, 3) (75000, 3) (75000, 3)
(25000, 3) (25000, 3) (25000, 3)
(75000,) (75000,) (75000,)
(25000,) (25000,) (25000,)


Данные разделены, соотношение соблюдается. 

### Вывод:

Данные подготовлены к обучению модели. Избыточные данные удалены, оставшиеся разделены на обучающиеся и валидационные выборки. 

На следующем этапе обучим модель линейной регрессии и проверим ее на валидационной выборке.

## Обучение и проверка модели

Обучим на обучающей выборке каждого региона модель линейной регрессии. Т. к. в этот раз у нас сразу три датафрейма, автоматизируем процесс обучения и проверки, создав специальную функцию.

In [74]:
def train_and_valid (region, features_train, features_valid, target_train, target_valid): #создаем функцию
    global rmse, r2, predictions_mean, predictions_region

    model = LinearRegression()
    model.fit(features_train, target_train)
    
    predictions_valid = model.predict(features_valid)
    predictions_mean = predictions_valid.mean()
        
    rmse = (mean_squared_error(target_valid, predictions_valid))**0.5
    r2 = (r2_score(target_valid, predictions_valid))
    
    mean_true = target_train.mean()
    predicted_valid = pd.Series(mean_true, index = target_valid)
    rmse_true = ((mean_squared_error(target_valid, predicted_valid)) ** 0.5).round(2)
    
    predictions_region = pd.Series(predictions_valid, index = target_valid.index)
               
    print (region)            
    print('Средний предсказанный запас сырья:', predictions_mean)
    print('Метрики линейной регрессии: RMSE', rmse, ', R2', r2)
    print()
    print('Проверка на адекватность предсказаний')
    print('Средний запас сырья:', mean_true, ', RMSE', rmse_true) 

In [75]:
#применяем функцию
train_and_valid('Первый регион:', features_train_one, features_valid_one, target_train_one, target_valid_one)
#переименовываем значимые переменные для дальнейшего использования
rmse_reg_1, r2_reg_1, predictions_mean_reg_1, predictions_region_1 = rmse, r2, predictions_mean, predictions_region

Первый регион:
Средний предсказанный запас сырья: 92.59256778438038
Метрики линейной регрессии: RMSE 37.5794217150813 , R2 0.27994321524487786

Проверка на адекватность предсказаний
Средний запас сырья: 92.64046775305694 , RMSE 44.29


In [76]:
train_and_valid('Второй регион:', features_train_two, features_valid_two, target_train_two, target_valid_two)
rmse_reg_2, r2_reg_2, predictions_mean_reg_2, predictions_region_2 = rmse, r2, predictions_mean, predictions_region

Второй регион:
Средний предсказанный запас сырья: 68.728546895446
Метрики линейной регрессии: RMSE 0.893099286775616 , R2 0.9996233978805127

Проверка на адекватность предсказаний
Средний запас сырья: 68.85895465854666 , RMSE 46.02


In [77]:
train_and_valid('Третий регион:', features_train_three, features_valid_three, target_train_three, target_valid_three)
rmse_reg_3, r2_reg_3, predictions_mean_reg_3, predictions_region_3 = rmse, r2, predictions_mean, predictions_region

Третий регион:
Средний предсказанный запас сырья: 94.96504596800489
Метрики линейной регрессии: RMSE 40.02970873393434 , R2 0.20524758386040443

Проверка на адекватность предсказаний
Средний запас сырья: 95.03858906371522 , RMSE 44.9


In [78]:
#сведем ключевые характеристики в таблицу для удобства восприятия и анализа
result = {'predictions_mean': [predictions_mean_reg_1, predictions_mean_reg_2, predictions_mean_reg_3],
    'rmse_score': [rmse_reg_1, rmse_reg_2, rmse_reg_3], 'r2_score':[r2_reg_1, r2_reg_2, r2_reg_3]}
results_of_learning = pd.DataFrame(data=result, index=('region 1', 'region 2', 'region 3'))
display(results_of_learning)

,predictions_mean,rmse_score,r2_score
region 1,92.592568,37.579422,0.279943
region 2,68.728547,0.893099,0.999623
region 3,94.965046,40.029709,0.205248


### Вывод:

Согласно результатам, полученным в ходе использования модели линейной регрессии, в первом и третьем регионах примерно равные запасы сырья, тогда как запасы второго региона существенно им уступают. 

Однако, значения метрик RMSE  и R2 у модели, примененной к данным второго региона, значительно лучше и максимально близки к идеальным. Впрочем, стоит учитывать, что на тестовой выборке результаты могут быть иными.

На следующем этапе подготовим данные к расчету прибыли, закрепив ключевые значения в отдельных переменных и произведя необходимые расчеты.

## Подготовка к расчёту прибыли

Для расчета прибыли нам потребуются значения бюджета на разработку скважин в каждом регионе, доход с каждой единицы продукта и количество скважин. Сохраним эти значения в отдельных переменных. 

Кроме того необходимо рассчитать достаточный объём сырья для безубыточной разработки новой скважины и сравнить его с объемами, имеющимися в каждом из рассматриваемых регионов.

In [79]:
#закрепляем ключевые значения
DEVELOPMENT_BUDGET = 10**10
UNIT_INCOME = 450000
NUMBER_OF_WELLS = 200

#считаем безубыточный объм сырья
enough_raw_material = (development_budget / (unit_income * number_of_wells))

#выводим запасы сырья на экран для сравнения
print('Достаточный объём сырья для безубыточной разработки', round(enough_raw_material, 3))
print('Средний объём сырья в первом регионе', region_one['product'].mean().round(3))
print('Средний объём сырья во втором регионе', region_two['product'].mean().round(3))
print('Средний объём сырья в третьем регионе', region_three['product'].mean().round(3))

Достаточный объём сырья для безубыточной разработки 111.111
Средний объём сырья в первом регионе 92.5
Средний объём сырья во втором регионе 68.825
Средний объём сырья в третьем регионе 95.0


### Вывод:

Согласно полученным результатам, ни в одном из регионов нет достаточных средних запасов для безубыточной разработки. Хуже всего, как уже отмечалось ранее, дела обстоят во втором регионе: средние запасы почти вдвое меньше необходимого "безубыточного" объема. Но и у региона с самым большим средним запасом (третий регион) сырья недостаточно (95 тыс. баррелей против 111.111 тыс.). 

Исходя из этого, для получения прибыли компании нужно выбрать самые перспективные для разработки скважины, т.е те, которые потенциально принесут больше прибыли при меньшем риске убытка. Такие скважины мы будем искать на следующем этапе исследования.

## Расчёт прибыли и рисков 

Выявим скважины с потенциально наиболее высокой прибылью и оценим риски убытков. Для начала создадим функцию, которая будет рассчитывать прибыль.

In [80]:
def profit(target, predictions):
    predictions_top_200 = pd.Series(predictions).sort_values(ascending=False)
    target_top_200 = target[predictions_top_200.index][:number_of_wells]
    profit = (target_top_200.sum() * unit_income) - development_budget
    return profit

Далее для подсчетов рисков и прибыли для каждого региона применим технику Bootstrap с 1000 выборок.

In [81]:
def bootstrap(prediction, target):
    
    state = np.random.RandomState(12345)
    values = []
    global final_results
    
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        probs_subsample = prediction[target_subsample.index]
        values.append(profit(target_subsample, probs_subsample))
        
    values = pd.Series(values)
    values_mean = (values.mean()) / (10**6)
    
    lower = ((values.quantile(0.025)) / (10**6)).round(2)
    upper = ((values.quantile(0.975)) / (10**6)).round(2)
    
    risk = 0
    
    for j in range(len(values)):
        if values[j] < 0:
            risk += 1
            
    risk = (risk / len(values)) * 100
    
    final_results = {'Средняя прибыль в млн руб.': values_mean, 'Начало доверительного интервала': lower,
                     'Конец доверительного интервала': upper, 'Риск убытков в %': risk}


In [82]:
#применяем технику Bootstrap к данным каждого региона
bootstrap(predictions_region_1, target_valid_one)
final_reg_1 = final_results

In [83]:
bootstrap(predictions_region_2, target_valid_two)
final_reg_2 = final_results

In [84]:
bootstrap(predictions_region_3, target_valid_three)
final_reg_3 = final_results

In [85]:
#сводим результаты в таблицу
results_final = final_reg_1, final_reg_2, final_reg_3
results_final = pd.DataFrame(data=results_final, index=('region 1', 'region 2', 'region 3'))
display(results_final)

,Средняя прибыль в млн руб.,Начало доверительного интервала,Конец доверительного интервала,Риск убытков в %
region 1,425.938527,-102.09,947.98,6.0
region 2,515.222773,68.87,931.55,1.0
region 3,435.008363,-128.88,969.71,6.4


### Вывод:

После подсчета прибыли и рисков наиболее перспективным для разработки выглядит второй регион:
- самая высокая средняя прибыль среди всех регионов (на втором месте третий регион с отрывом в 80 млн руб, на последнем - первый регион с отрывом в 89 млн);
- наименьший уровень риска (в 6 раз меньше других рассматриваемых регионов);
- с 95% вероятностью даже наименее прибыльные из выбранных скважин второго региона не принесут убыток, остановившись на примерной отметке средней прибыли в 68.87 млн руб.

На этом наше исследование можно считать завершенным. Перейдем к итоговым выводам.

## Итоговые выводы

После проведенного исследования потенциально прибыльных для разработки новых месторождений регионов с использованием машинного обучения, можно сделать вывод, что выгоднее всего бурить новые скважины во втором регионе:
- высокие значения метрик RMSE и R2 у модели, примененной к валидационной выборке данных второго региона;
- самая высокая средняя прибыль;
- уровень риска получения убытка менее 2.5%.

Исходя из всего выше сказанного, регион, где добыча принесёт наибольшую прибыль при наименьшем риске убытков - второй.